In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re

df = pd.read_csv("FusionTech Online Reviews - FusionTech Online Reviews.csv")  # assuming you have a CSV
df['cleaned'] = df['text'].astype(str).str.lower().apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))


In [4]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

from deep_translator import GoogleTranslator

def translate_to_english(text):
    try:
        return GoogleTranslator(source='auto', target='en').translate(text)
    except:
        return text  # fallback if translation fails

df['translated'] = df['text'].astype(str).apply(translate_to_english)


vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['cleaned'])

lda = LatentDirichletAllocation(n_components=5, random_state=42)
topics = lda.fit_transform(X)
df['topic'] = topics.argmax(axis=1)

In [5]:
import pandas as pd
from collections import Counter
import re
import nltk

# Make sure your DataFrame 'df' is loaded and has a 'cleaned' column
# For example:
# df = pd.read_csv("FusionTech Online Reviews - FusionTech Online Reviews.csv")
# df['cleaned'] = df['text'].astype(str).str.lower().apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))

# Tokenize the cleaned reviews
all_words = ' '.join(df['cleaned']).split()

nltk.download('stopwords')
from nltk.corpus import stopwords

# Define stop words (you can extend this list as needed)
stop_words = set(stopwords.words('english'))

custom_stopwords = {'for', 'to', 'a', 'the', 'i', 'im'}

# Combine the default and custom stop words
all_stopwords = stop_words.union(custom_stopwords)

# Filter out the stop words using the combined set
filtered_words = [word for word in all_words if word not in all_stopwords]

# Count frequency of each word from the filtered list
word_counts = Counter(filtered_words)

# Display the 20 most common keywords after filtering
print("Top 20 most common keywords (after filtering):")
for word, count in word_counts.most_common(20):
    print(f"{word}: {count}")

# Define a list of keywords associated with faulty traits
faulty_keywords = ['overheating', 'defective', 'malfunction', 'broken', 'issue', 
                   'problem', 'faulty', 'hazard', 'fire', 'slow','awful', 'failure']

print("\nFaulty trait keywords frequency:")
for keyword in faulty_keywords:
    count = word_counts.get(keyword, 0)
    print(f"{keyword}: {count}")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Narel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Top 20 most common keywords (after filtering):
laptop: 6100
br: 4561
fusiontech: 2131
screen: 1936
computer: 1869
great: 1714
good: 1665
like: 1662
one: 1609
gaming: 1523
get: 1471
use: 1446
would: 1375
games: 1182
battery: 1128
really: 1080
keyboard: 1046
even: 1015
also: 1012
well: 994

Faulty trait keywords frequency:
overheating: 50
defective: 72
malfunction: 10
broken: 79
issue: 715
problem: 563
faulty: 37
hazard: 1
fire: 11
slow: 153
awful: 54
failure: 42


In [6]:
# Define a dictionary mapping faulty keywords to response templates
response_templates = {
    'overheating': (
        "It appears you're experiencing overheating issues. "
        "Please ensure that your device is well-ventilated and not exposed to extreme temperatures. "
        "If the problem persists, contact our support team for further assistance."
    ),
    'defective': (
        "We're sorry to hear that your product appears defective. "
        "Please reach out to our customer service team to arrange a repair or replacement."
    ),
    'malfunction': (
        "It sounds like your device is malfunctioning. "
        "We recommend restarting it, and if the issue continues, contacting our support team for further help."
    ),
    'broken': (
        "It seems that something may be broken. "
        "Please contact our support team so we can help arrange a repair or a replacement."
    ),
    'issue': (
        "We noticed you're experiencing an issue with our product. "
        "Could you please provide more details so we can help address it effectively?"
    ),
    'problem': (
        "We apologize for the problem you're facing. "
        "Please contact our customer service for troubleshooting and further assistance."
    ),
    'faulty': (
        "Our records indicate a faulty product. "
        "Please get in touch with our support team immediately to arrange a repair or replacement."
    )
}

# Function to generate a response for a given review text
def generate_response(review_text):
    review_lower = review_text.lower()  # ensure case-insensitive matching
    # Loop through each faulty keyword and return the corresponding response
    for keyword, response in response_templates.items():
        if keyword in review_lower:
            return response
    # If no keywords are found, return a default message
    return "Thank you for your feedback! If you have any further concerns, please don't hesitate to contact us."

# Example usage:
# Assuming you have a DataFrame 'df' with a 'text' column containing the reviews:
sample_review = df['text'].iloc[0]  # taking the first review as an example
response = generate_response(sample_review)
print("Review:", sample_review)
print("Response:", response)


Review: I have been using laptops for the past 30 years, and haven’t bought a new one since 2013.  It was a special treat getting this new FusionTech G3. I’m not a crazy gamer, but I do play games. I was attracted to this computer because I wanted performance, and I know gamers expect high performance!<br />PROS:<br />-FAST FAST FAST. It is by far the fastest computer I’ve ever owned. I often blamed my speed on my Internet connection -- but am now discovering that my computer itself was at least 75% of the  problem.<br />-The graphics card is amazing!  My 10yr old nephew tried out Minecraft, Java Edition on my laptop. He gave it a 98/100.  The Frames Per Second rate started around 90+.  He gave it the TNT test (a fast animation to stress out the frame rate) and the frame rate dropped 30 then immediately recovered to the high level.<br />-The keyboard glows blue. Not only is it super cool, it’s easy to see in low light.<br />-Compared to other laptops I’ve had, this one is MUCH cooler (

In [7]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the product manual from a text file.
# Make sure you have a file named "product_manual.txt" with your manual's text.
with open("Manual.txt", "r", encoding="utf-8") as f:
    manual_text = f.read()

# Split the manual into paragraphs (assuming paragraphs are separated by double newlines)
paragraphs = manual_text.split("\n\n")
# Clean paragraphs: strip extra whitespace and remove empty ones.
paragraphs = [p.strip() for p in paragraphs if p.strip()]

# Create a TF-IDF vectorizer; you can add custom stop words if needed.
vectorizer = TfidfVectorizer(stop_words="english")
manual_vectors = vectorizer.fit_transform(paragraphs)

def get_manual_response(review_text):
    """
    Given a review text, vectorize it and compute cosine similarity
    with each paragraph from the manual. Returns the most similar paragraph.
    """
    # Optionally, clean the review_text if needed
    review_text = review_text.lower()
    review_text = re.sub(r'[^a-zA-Z\s]', '', review_text)
    
    review_vector = vectorizer.transform([review_text])
    similarities = cosine_similarity(review_vector, manual_vectors)
    best_index = similarities.argmax()
    return paragraphs[best_index]

# Example usage:
review_example = "I'm experiencing issues with overheating and the device shuts down unexpectedly."
response_from_manual = get_manual_response(review_example)
print("Response from manual:")
print(response_from_manual)


Response from manual:
Wi-Fi power cycle
About this task
If your computer is unable to access the Internet due to Wi-Fi connectivity issues, reset your Wi-Fi device by performing the
following steps:
Steps
1. Turn off the computer.
2. Turn off the modem.
NOTE: Some Internet service providers (ISPs) provide a modem and router combo device.
3. Turn off the wireless router.
4. Wait for 30 seconds.
5. Turn on the wireless router.
6. Turn on the modem.
7. Turn on the computer.
